In [33]:
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
# text preprocessing modules
from string import punctuation
# text preprocessing modules
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import regex as re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pritish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pritish\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
df = pd.read_excel(r"Agro.xlsx")

In [35]:
df.columns

Index(['Unnamed: 0', 'Title', 'labels'], dtype='object')

In [36]:
df1 = df[['Title', 'labels']].copy()

In [37]:
df1.head()

,Title,labels
0,Pesticidally active cyclic amine compounds,insecticide
1,Pest control composition and control method,insecticide
2,Method for controlling pests,insecticide
3,Compositions for controlling deleterious arthr...,insecticide
4,Insect Control Composition,insecticide


In [38]:
pd.DataFrame(df1.labels.unique()).values

array([['insecticide'],
       ['herbicide'],
       ['fungicide'],
       ['miscellaneous']], dtype=object)

In [39]:
# Create a new column 'category_id' with encoded categories 
df1['category_id'] = df1['labels'].factorize()[0]
category_id_df = df1[['labels', 'category_id']].drop_duplicates()

In [40]:
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'labels']].values)
# New dataframe
df1.head()

,Title,labels,category_id
0,Pesticidally active cyclic amine compounds,insecticide,0
1,Pest control composition and control method,insecticide,0
2,Method for controlling pests,insecticide,0
3,Compositions for controlling deleterious arthr...,insecticide,0
4,Insect Control Composition,insecticide,0


In [41]:
df1['category_id'].value_counts()

0    1192
2    1112
1     541
3     422
Name: category_id, dtype: int64

In [42]:
stop_words =  stopwords.words('english')
def text_cleaning(text, remove_stop_words=True, lemmatize_words=True):
    # Clean the text, with the option to remove stop_words and to lemmatize word
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) # remove numbers
        
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if lemmatize_words:
        text = text.split()
        lemmatizer = WordNetLemmatizer() 
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(lemmatized_words)
    
    # Return a list of words
    return(text)

In [43]:
df1["cleaned_review"] = df1["Title"].apply(text_cleaning)

In [44]:
#split features and target from  data 
X = df1["cleaned_review"]
y = df1.category_id.values

In [45]:
# split data into train and validate
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.40,
    random_state=42,
    shuffle=True,
    stratify=y,
)

In [46]:
# Create a classifier in pipeline
sentiment_classifier = Pipeline(steps=[
                               ('pre_processing',TfidfVectorizer(lowercase=False)),
                                 ('linear_svc',LinearSVC())
                                 ])

In [47]:
 # train the sentiment classifier 
sentiment_classifier.fit(X_train,y_train)

Pipeline(steps=[('pre_processing', TfidfVectorizer(lowercase=False)),
                ('linear_svc', LinearSVC())])

In [48]:
# test model performance on valid data 
y_preds = sentiment_classifier.predict(X_valid)

In [49]:
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    
)
accuracy_score(y_valid,y_preds)

0.8026013771996939

In [50]:
#save model 
import joblib 
joblib.dump(sentiment_classifier, 'multiclass_model_pipeline2.pkl')

['multiclass_model_pipeline2.pkl']